In [1]:
#Student Name: Zequn Yuan
#Course Name: CS-505 - 740
# predictive_maintenance.ipynb

# 1. Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [12]:
# 2. Load Dataset
import pandas as pd

# Load dataset with whitespace delimiter
df = pd.read_csv('C:/Users/Yuan/Downloads/train_FD001.txt', delim_whitespace=True, header=None)

# Assign column names
df.columns = ['engine_id', 'cycle'] + ['op_setting1', 'op_setting2', 'op_setting3'] + [f'sensor{i}' for i in range(1, 22)]

# Check the result
print(df.head())
print(f"Shape: {df.shape}")

   engine_id  cycle  op_setting1  op_setting2  op_setting3  sensor1  sensor2  \
0          1      1      -0.0007      -0.0004        100.0   518.67   641.82   
1          1      2       0.0019      -0.0003        100.0   518.67   642.15   
2          1      3      -0.0043       0.0003        100.0   518.67   642.35   
3          1      4       0.0007       0.0000        100.0   518.67   642.35   
4          1      5      -0.0019      -0.0002        100.0   518.67   642.37   

   sensor3  sensor4  sensor5  ...  sensor12  sensor13  sensor14  sensor15  \
0  1589.70  1400.60    14.62  ...    521.66   2388.02   8138.62    8.4195   
1  1591.82  1403.14    14.62  ...    522.28   2388.07   8131.49    8.4318   
2  1587.99  1404.20    14.62  ...    522.42   2388.03   8133.23    8.4178   
3  1582.79  1401.87    14.62  ...    522.86   2388.08   8133.83    8.3682   
4  1582.85  1406.22    14.62  ...    522.19   2388.04   8133.80    8.4294   

   sensor16  sensor17  sensor18  sensor19  sensor20  sen

C:\Users\Yuan\AppData\Local\Temp\ipykernel_6412\643613516.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('C:/Users/Yuan/Downloads/train_FD001.txt', delim_whitespace=True, header=None)


In [13]:
# 3. Preprocessing
df['RUL'] = df.groupby('engine_id')['cycle'].transform(max) - df['cycle']
selected_sensors = ['sensor2', 'sensor3', 'sensor4', 'sensor7', 'sensor8', 'sensor11', 'sensor15', 'sensor20']
data = df[selected_sensors + ['RUL']]

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

C:\Users\Yuan\AppData\Local\Temp\ipykernel_6412\971533444.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df['RUL'] = df.groupby('engine_id')['cycle'].transform(max) - df['cycle']


In [14]:
# 4. Train/Test Split
X = scaled_data[:, :-1]
y = (df['RUL'] < 30).astype(int)  # Binary classification

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
# 5. Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest Report:")
print(classification_report(y_test, y_pred_rf))


NameError: name 'X_train' is not defined

In [6]:
# 6. LSTM Preparation
def create_sequences(X, y, time_steps=30):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i+time_steps])
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X, y)

X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(X_seq, y_seq, test_size=0.2)


NameError: name 'X' is not defined

In [7]:
# 7. LSTM Model
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(30, X.shape[1])))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=64, validation_data=(X_test_seq, y_test_seq))


NameError: name 'X' is not defined